In [ ]:
import numpy as np
import cv2
from scipy import stats


def load_movie_data(filepath):
    # Dummy function to represent loading your specific dataset
    # Replace this with actual code to load your movie data
    return np.load(filepath)


def preprocess_frames(frames):
    # Assuming the first few frames can serve as a baseline for normalization
    normalizing_frame = np.median(frames[:10], axis=0)
    normalized_frames = frames / (normalizing_frame + 1e-6)  # Prevent division by zero
    return normalized_frames


def difference_and_threshold(normalized_frames, threshold=0.1):
    # Custom threshold based on expected Ca²⁺ event intensities
    diff_frames = np.diff(normalized_frames, axis=0)
    thresholded_frames = (diff_frames > threshold).astype(np.uint8)
    return thresholded_frames


def find_spark_coordinates(thresholded_frames):
    # Simplified for demonstration; real implementation may require more sophisticated analysis
    coordinates = np.argwhere(thresholded_frames)
    return coordinates


def statistical_classification(coordinates, frames, pre_frames=10, alpha=0.05):
    true_sparks = []
    for idx, y, x in coordinates:
        if idx < pre_frames:
            continue  # Not enough frames for analysis
        pre_fluorescence = frames[idx - pre_frames : idx, y, x].flatten()
        m, s = np.mean(pre_fluorescence), np.std(pre_fluorescence)
        spark_intensity = frames[idx, y, x]
        t_score = (spark_intensity - m) / (s if s > 0 else 1)
        p_value = stats.t.sf(np.abs(t_score), pre_frames - 1)  # Two-sided test
        if p_value < alpha:
            true_sparks.append((x, y, idx))
    return true_sparks


# Example usage
# filepath = 'path/to/your/movie/data.npy'
# frames = load_movie_data(filepath)
# Ensure your data is loaded such that frames is a numpy array of shape (duration, 64, 512)

frames = np.random.rand(600, 64, 512)  # Dummy data for demonstration
normalized_frames = preprocess_frames(frames)
thresholded_frames = difference_and_threshold(normalized_frames)
coordinates = find_spark_coordinates(thresholded_frames)
true_sparks = statistical_classification(coordinates, normalized_frames)

print(f"Detected {len(true_sparks)} true Ca²⁺ sparks.")